In [1]:
import random
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import esda
import libpysal.weights as weights
from esda.moran import Moran
from shapely.geometry import Point, MultiPoint, LineString, Polygon, shape
import json
import pylab
import libpysal
import numpy as np
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import f1_score
from pyclustering.cluster.cure import cure
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from sklearn import preprocessing

In [2]:
def permutation(lst):
    if len(lst) == 0:
        return []

    if len(lst) == 1:
        return [lst]

    l = []
    for i in range(len(lst)):
        m = lst[i]
        remLst = lst[:i] + lst[i+1:]
        for p in permutation(remLst):
            l.append([m] + p)       
    return l

In [3]:
def get_f1_score(df, permut):
    def match_clus(x, permut):
        if x == 0:
            return int(permut[0])
        elif x == 1:
            return int(permut[1])
        elif x == 2:
            return int(permut[1])
        else:
            return x

    df["group_match"] = df["group"].apply(lambda x: match_clus(x, permut))
    return df, f1_score(df.group_match.values, df.clus_group_gt.values, average='macro')

In [4]:
def get_max_f1_score(df):
    max_f1 = 0
    max_p = []
    for p in permutation([1,3,4]):
        df, f1 = get_f1_score(df, p)
        if max_f1 < f1:
            max_f1 = f1
            max_p = p
    print("f1_score ", max_f1, max_p)

In [5]:
def cal_joint_statistic(nyc_data, w_voronoi):
    matched_connects = 0
    all_neighbors_connects = 0
    for obj_id, neighbors in w_voronoi.neighbors.items():
        obj_clus = nyc_data.iat[obj_id, -1]
        for nei in neighbors:
            nei_clus = nyc_data.iat[nei, -1]
            all_neighbors_connects += 1
            if obj_clus == nei_clus:
                matched_connects += 1
    return matched_connects / all_neighbors_connects

In [6]:
nyc_check_in = gpd.read_file('data/nyc_checkin.shp')
nyc_check_in.head(1)

,venueId,userId,gender,friend_num,follow_num,latitude,longitude,venueCateg,week,hour,geometry
0,3fd66200f964a52000e71ee3,445,male,4.0,13.0,40.73385,-74.002998,Jazz Club,Sat,8,POINT (-74.00300 40.73385)


In [7]:
nyc_check_in.groupby("venueCateg").count().sort_values("venueId").tail(1)

,venueId,userId,gender,friend_num,follow_num,latitude,longitude,week,hour,geometry
venueCateg,,,,,,,,,,
Subway,10042,10042,10042,10042,10042,10042,10042,10042,10042,10042


In [8]:
venueCateg_list = ["Gym", "Office", "Home (private)"]
venueId_list = pd.DataFrame(nyc_check_in.venueId.unique()).sample(frac=0.3).values.squeeze()
nyc_check_sticc = nyc_check_in[(nyc_check_in.venueCateg.isin(venueCateg_list))&(nyc_check_in.venueId.isin(venueId_list))]
print(nyc_check_sticc.shape)
nyc_check_sticc.head(1)

(5909, 11)


,venueId,userId,gender,friend_num,follow_num,latitude,longitude,venueCateg,week,hour,geometry
1828,3fd66200f964a5206fe71ee3,654,male,103.0,46.0,40.752901,-73.974176,Gym,Mon,17,POINT (-73.97418 40.75290)


In [9]:
def return_week(x):
    if x == "Mon":
        return 1
    elif x == "Tue":
        return 2
    elif x == "Wed":
        return 3
    elif x == "Thu":
        return 4
    elif x == "Fri":
        return 5
    elif x == "Sat":
        return 6
    elif x == "Sun":
        return 7
    
def return_category(x):
    if x == "Gym":
        return 1
    elif x == "Coffee Shop":
        return 2
    elif x == "Office":
        return 3
    elif x == "Home (private)":
        return 4
    elif x == "Subway":
        return 5

nyc_check_sticc["week_attr"] = nyc_check_sticc["week"].apply(lambda x: return_week(x))
nyc_check_sticc["category"] = nyc_check_sticc["venueCateg"].apply(lambda x: return_category(x))
nyc_check_sticc = nyc_check_sticc.reset_index().drop("index", axis=1)

/home/kangyuhao/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/home/kangyuhao/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [10]:
nyc_check_sticc.head(1)

,venueId,userId,gender,friend_num,follow_num,latitude,longitude,venueCateg,week,hour,geometry,week_attr,category
0,3fd66200f964a5206fe71ee3,654,male,103.0,46.0,40.752901,-73.974176,Gym,Mon,17,POINT (-73.97418 40.75290),1,1


In [11]:
kd = libpysal.cg.KDTree(np.array(nyc_check_sticc[["latitude", "longitude"]].values))
wnn = libpysal.weights.KNN(kd, 3)

/home/kangyuhao/anaconda3/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 140 disconnected components.
  warnings.warn(message)


In [12]:
nearest_pt = pd.DataFrame().from_dict(wnn.neighbors, orient="index")
for i in range(nearest_pt.shape[1]):
    nearest_pt = nearest_pt.rename({i:f"n_pt_{i}"}, axis=1)
nearest_pt.head(1)

,n_pt_0,n_pt_1,n_pt_2
0,3556,9,22


In [13]:
nyc_check_sticc = nyc_check_sticc.join(nearest_pt)
nyc_check_sticc.head(1)

,venueId,userId,gender,friend_num,follow_num,latitude,longitude,venueCateg,week,hour,geometry,week_attr,category,n_pt_0,n_pt_1,n_pt_2
0,3fd66200f964a5206fe71ee3,654,male,103.0,46.0,40.752901,-73.974176,Gym,Mon,17,POINT (-73.97418 40.75290),1,1,3556,9,22


In [14]:
nyc_check_sticc[["week_attr", "hour", "n_pt_0", "n_pt_1", 
                 "n_pt_2"]].to_csv(r'nyc_checkin3.txt', header=None, index=True, sep=',')

In [15]:
w_voronoi = weights.Voronoi.from_dataframe(nyc_check_sticc)

# STICC

In [16]:
!python TICC_main.py --fname=nyc_checkin3.txt --oname=result_nyc_checkin3.txt --attr_idx_start=1 \
--attr_idx_end=2 --spatial_idx_start=3 --spatial_idx_end=5 \
--spatial_radius 4 --number_of_clusters 3 --lambda_parameter 10e-2 --beta 5 --maxIters 20

lam_sparse 0.1
switch_penalty 5.0
num_cluster 3
num stacked 4
completed getting the data
2 (5909, 2) (5909, 3)



ITERATION ### 0
OPTIMIZATION for Cluster # 0 DONE!!!
OPTIMIZATION for Cluster # 1 DONE!!!
OPTIMIZATION for Cluster # 2 DONE!!!
length of the cluster  0 ------> 3087
length of the cluster  1 ------> 1475
length of the cluster  2 ------> 1347
UPDATED THE OLD COVARIANCE
beginning the smoothening ALGORITHM
length of cluster # 0 --------> 3196
length of cluster # 1 --------> 1611
length of cluster # 2 --------> 1102
Done writing the figure







ITERATION ### 1
OPTIMIZATION for Cluster # 0 DONE!!!
OPTIMIZATION for Cluster # 1 DONE!!!
OPTIMIZATION for Cluster # 2 DONE!!!
length of the cluster  0 ------> 3196
length of the cluster  1 ------> 1611
length of the cluster  2 ------> 1102
UPDATED THE OLD COVARIANCE
beginning the smoothening ALGORITHM
length of cluster # 0 --------> 3012
length of cluster # 1 --------> 1478
length of cluster # 2 --------> 1419
Done writing the figure



In [17]:
group = pd.read_table('result_nyc_checkin3.txt', names=["group"])
result_nyc_check_sticc = nyc_check_sticc.join(group)
result_nyc_check_sticc = result_nyc_check_sticc.rename({"category": "clus_group_gt"}, axis=1)
print("Adjusted rand score", adjusted_rand_score(result_nyc_check_sticc["group"].values, 
                                                 result_nyc_check_sticc.clus_group_gt.values))
sp_contiguity = cal_joint_statistic(result_nyc_check_sticc, w_voronoi)
print("Spatial contiguity: ", sp_contiguity)
get_max_f1_score(result_nyc_check_sticc)

Adjusted rand score 0.299048156707623
Spatial contiguity:  0.7245619074978454
f1_score  0.5239363875462164 [3, 4, 1]


# Other methods

In [18]:
def get_pycluster_result(ground_truth, cluster_method):
#     data = ground_truth[["week_attr", "hour"]].values # For K-Means
    data = ground_truth[["week_attr", "hour", "latitude", "longitude"]].values # For Sp K-Means

    if cluster_method == kmeans:
        initial_centers = kmeans_plusplus_initializer(data.tolist(), 2).initialize()
        instance = cluster_method(data.tolist(), initial_centers)
    elif cluster_method == cure:
        print("cure")
        instance = cure(data, 3)
    else:
        instance = cluster_method(data.tolist(), 2)

    instance.process()
    clusters = instance.get_clusters()
    
    clusters_result = []
    for i, clus in enumerate(clusters):
        for data in clus:
            clusters_result.append([data, i])
    clusters_result_df = pd.DataFrame(clusters_result, columns=["pt", "group"]).sort_values("pt").set_index("pt")
    return clusters_result_df

# K-Means

In [19]:
group = get_pycluster_result(nyc_check_sticc, kmeans)
result_nyc_check_sticc = nyc_check_sticc.join(group)
result_nyc_check_sticc = result_nyc_check_sticc.rename({"category": "clus_group_gt"}, axis=1)
print("Adjusted rand score", adjusted_rand_score(result_nyc_check_sticc["group"].values, 
                                                 result_nyc_check_sticc.clus_group_gt.values))
sp_contiguity = cal_joint_statistic(result_nyc_check_sticc, w_voronoi)
print("Spatial contiguity: ", sp_contiguity)
get_max_f1_score(result_nyc_check_sticc)

Adjusted rand score 0.06540493878619441
Spatial contiguity:  0.6700948003447286
f1_score  0.38086125317189695 [3, 4, 1]


# Sp K-Means

In [20]:
group = get_pycluster_result(nyc_check_sticc, kmeans)
result_nyc_check_sticc = nyc_check_sticc.join(group)
result_nyc_check_sticc = result_nyc_check_sticc.rename({"category": "clus_group_gt"}, axis=1)
print("Adjusted rand score", adjusted_rand_score(result_nyc_check_sticc["group"].values, 
                                                 result_nyc_check_sticc.clus_group_gt.values))
sp_contiguity = cal_joint_statistic(result_nyc_check_sticc, w_voronoi)
print("Spatial contiguity: ", sp_contiguity)
get_max_f1_score(result_nyc_check_sticc)

Adjusted rand score 0.06540493878619441
Spatial contiguity:  0.6700948003447286
f1_score  0.38086125317189695 [3, 4, 1]


# CURE

In [21]:
group = get_pycluster_result(nyc_check_sticc, cure)
result_nyc_check_sticc = nyc_check_sticc.join(group)
result_nyc_check_sticc = result_nyc_check_sticc.rename({"category": "clus_group_gt"}, axis=1)
print("Adjusted rand score", adjusted_rand_score(result_nyc_check_sticc["group"].values, 
                                                 result_nyc_check_sticc.clus_group_gt.values))
sp_contiguity = cal_joint_statistic(result_nyc_check_sticc, w_voronoi)
print("Spatial contiguity: ", sp_contiguity)
get_max_f1_score(result_nyc_check_sticc)

cure
Adjusted rand score 0.0729293684699148
Spatial contiguity:  0.6272335535765584
f1_score  0.4208030109481018 [3, 4, 1]


# GMM

In [22]:
from sklearn.mixture import GaussianMixture

In [23]:
gmm_data = nyc_check_sticc.copy()
gmm_data.head(1)

,venueId,userId,gender,friend_num,follow_num,latitude,longitude,venueCateg,week,hour,geometry,week_attr,category,n_pt_0,n_pt_1,n_pt_2
0,3fd66200f964a5206fe71ee3,654,male,103.0,46.0,40.752901,-73.974176,Gym,Mon,17,POINT (-73.97418 40.75290),1,1,3556,9,22


In [24]:
X = gmm_data[['hour', 'week_attr']].values

In [25]:
gm = GaussianMixture(n_components=3).fit(X)
gmm = pd.DataFrame(gm.predict(X), columns=["group"])
gmm.head(1)

,group
0,1


In [26]:
result_nyc_check_sticc = nyc_check_sticc.join(gmm)
result_nyc_check_sticc = result_nyc_check_sticc.rename({"category": "clus_group_gt"}, axis=1)
print("Adjusted rand score", adjusted_rand_score(result_nyc_check_sticc["group"].values, 
                                                 result_nyc_check_sticc.clus_group_gt.values))
sp_contiguity = cal_joint_statistic(result_nyc_check_sticc, w_voronoi)
print("Spatial contiguity: ", sp_contiguity)
get_max_f1_score(result_nyc_check_sticc)

Adjusted rand score 0.09072443404391904
Spatial contiguity:  0.6405630565929331
f1_score  0.4349576813996136 [3, 4, 1]
